# Отслеживание ошибок

Для отладки программы обычно используются принты или встроенный в IDE дебаг

Такой способ имеет место быть в небольших и простых проектах

## Сложность реальных приложений

Возьмем для примера типичный сайт. Что он в себя включает:

+ DNS. Система трансляции имени сайта в ip-адрес сервера.
+ Веб-сервер. Программа, обслуживающая входящие запросы, перенаправляет их в код приложения и забирает от приложения данные для пользователей.
+ Физический сервер (или виртуальный) с его окружением. Включает в себя операционную систему, установленные и запущенные обслуживающие программы, например, мониторинг.
+ База данных. Внешнее хранилище, с которым связывается код приложения и обменивается информацией.
+ Само приложение. Помимо кода, который пишут программисты, приложение включает в себя сотни тысяч и миллионы строк кода сторонних библиотек. Кроме этого, код работает внутри фреймворка, у которого свои собственные правила обработки входящих запросов.
+ Фронтенд часть. Код, который выполняется в браузере пользователя. И системы сборки для разработки, например, Webpack.

И это только самый простой случай. Реальность же значительно сложнее: множество разноплановых серверов, системы кеширования (ускорения доступа), асинхронный код, очереди, внешние сервисы, облачные сервисы. Все это выглядит как многослойный пирог, внутри которого где-то работает написанный нами код. И этот код составляет лишь небольшую часть всего происходящего. Как в такой ситуации понять, на каком этапе был сбой, или все пошло не по плану? Для этого, как минимум, нужно определить, в каком слое произошла ошибка. Но даже это не самое сложное. Об ошибках в работающем приложении узнают не сразу, а уже потом, — когда ошибка случилась и, иногда, больше не воспроизводится.

## Логирование

И для всего этого многообразия систем существует единое решение — логирование. В простейшем случае логирование сводится к файлу на диске, куда разные программы записывают (логируют) свои действия во время работы. Такой файл называют логом или журналом. Как правило, внутри лога одна строчка соответствует одному действию.

```173.245.52.110 - [19/Jan/2021:01:54:20 +0000] "GET /my HTTP/1.1" 200 46018
108.162.219.13 - [19/Jan/2021:01:54:20 +0000] "GET /sockjs-node/244/gdt1vvwa/websocket HTTP/1.1" 0 0
162.158.62.12 - [19/Jan/2021:01:54:20 +0000] "GET /packs/css/application.css HTTP/1.1" 304 0
162.158.62.84 - [19/Jan/2021:01:54:20 +0000] "GET /packs/js/runtime-eb0a99abbe8cf813f110.js HTTP/1.1" 304 0
108.162.219.111 - [19/Jan/2021:01:54:20 +0000] "GET /packs/js/application-2cba5619945c4e5946f1.js HTTP/1.1" 304 0
108.162.219.21 - [19/Jan/2021:01:54:20 +0000] "GET /packs/js/0564a7b5d773bab52e53.js HTTP/1.1" 304 0
108.162.219.243 - [19/Jan/2021:01:54:20 +0000] "GET /packs/js/6fb7e908211839fac06e.js HTTP/1.1" 304 0

Выше небольшой кусок лога веб-сервера Хекслета. Из него видно ip-адрес, с которого выполнялся запрос на страницу и какие ресурсы загружались, метод HTTP, ответ бекенда (кода) и размер тела ответа в HTTP. Очень важно наличие даты. 

Многие программы логируют прямо в консоль, например Webpack показывает процесс и результаты сборки:

```｢wds｣: Project is running at http://hexletdev4.com/
｢wds｣: webpack output is served from /packs/
｢wds｣: Content not from webpack is served from /root/hexlet/public/packs
｢wds｣: 404s will fallback to /index.html
｢wdm｣: assets by chunk 10.8 MiB (auxiliary name: application) 115 assets
sets by path js/ 13.8 MiB
assets by path js/*.js 13.8 MiB 52 assets
assets by path js/pages/*.js 5.1 KiB
  asset js/pages/da223d3affe56711f31f.js 2.6 KiB [emitted] [immutable] (name: pages/my_learning) 1 related asset
  asset js/pages/04adacfdd660803b19f1.js 2.5 KiB [emitted] [immutable] (name: pages/referral) 1 related asset
sets by chunk 9.14 KiB (auxiliary id hint: vendors)

### Уровни логирования

Чем больше информации выводится в логах, тем лучше и проще отладка, но когда данных слишком много, то в них тяжело искать нужное. В особо сложных случаях логи могут генерироваться с огромной скоростью и в гигантских размерах. Работать в такой ситуации нелегко. Чтобы как-то сгладить ситуацию, системы логирования вводят разные уровни. Обычно это:

+ debug - самый низкий уровень логирования, предназначенный для отладочных сообщений, для вывода диагностической информации о приложении
+ info - этот уровень предназначен для вывода данных о фрагментах кода, работающих так, как ожидается
+ warning - этот уровень логирования предусматривает вывод предупреждений, он применяется для записи сведений о событиях, на которые программист обычно обращает внимание. Такие события вполне могут привести к проблемам при работе приложения
+ error - этот уровень логирования предусматривает вывод сведений об ошибках — о том, что часть приложения работает не так как ожидается, о том, что программа не смогла правильно выполниться
+ critical - этот уровень используется для вывода сведений об очень серьёзных ошибках, наличие которых угрожает нормальному функционированию всего приложения. Если не исправить такую ошибку — это может привести к тому, что приложение прекратит работу


### Типы логов

Для удобной работы с логами их делят на типы. Это помогает быстрее находить нужные и выбирать правильные инструменты для работы с ними. Например, выделяют:

+ системные логи, то есть те, которые связаны с системными событиями
+ серверные логи, регистрирующие обращения к серверу и возникшие при этом ошибки
+ логи баз данных, фиксирующие запросы к базам данных
+ почтовые логи, относящиеся к входящим/исходящим письмам и отслеживающие ошибки, из-за которых письма не были доставлены
+ логи авторизации
+ логи аутентификации
+ логи приложений, установленных на этих операционных системах


### Логирование в Python

В Python имеется встроенный модуль logging, применяемый для решения задач логирования. 

In [1]:
import logging

Встроенный модуль логирования Python даёт нам простой в использовании функционал и предусматривает пять уровней логирования

In [2]:
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


Как видите, сообщения, выведенные с уровнями логирования warning, error и critical, попадают в консоль. А сообщения с уровнями debug и info — не попадают.

Это так из-за того, что в консоль выводятся лишь сообщения с уровнями от warning и выше. Но это можно изменить, настроив логгер и указав ему, что в консоль надо выводить сообщения, начиная с некоего, заданного вами, уровня логирования.

Подобный подход к логированию, когда данные выводятся в консоль, не особо лучше использования print(). На практике может понадобиться записывать логируемые сообщения в файл. Этот файл будет хранить данные и после того, как работа программы завершится. Такой файл можно использовать в качестве журнала отладки.

#### Логирование в файл

Для того чтобы настроить простую систему логирования в файл — можно воспользоваться конструктором basicConfig(). Вот как это выглядит:

In [2]:
logging.basicConfig(level=logging.INFO, filename="py_log.log", filemode="w", force=True)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Поговорим о логгере root, рассмотрим параметры basicConfig():

+ level: это — уровень, на котором нужно начинать логирование. Если он установлен в info — это значит, что все сообщения с уровнем debug игнорируются.
+ filename: этот параметр указывает на объект обработчика файла. Тут можно указать имя файла, в который нужно осуществлять логирование.
+ filemode: это — необязательный параметр, указывающий режим, в котором предполагается работать с файлом журнала, заданным параметром filename. Установка filemode в значение w (write, запись) приводит к тому, что логи перезаписываются при каждом запуске модуля. По умолчанию параметр filemode установлен в значение a (append, присоединение), то есть — в файл будут попадать записи из всех сеансов работы программы.
+ force: удаление корневых обработчиков

Помимо базовой информации, выводимой в лог, может понадобится снабдить записи отметками времени, указывающими на момент вывода той или иной записи. Это упрощает анализ логов. Сделать это можно, воспользовавшись параметром конструктора format:

In [3]:
logging.basicConfig(level=logging.INFO, filename="py_log.log", filemode="w", format="%(asctime)s %(levelname)s %(message)s", force=True)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Больше форматов: https://docs.python.org/3/library/logging.html#logrecord-attributes

#### Логирование значений переменных и исключений

Исключительные ситуации или исключения (exceptions) – это ошибки, обнаруженные при исполнении. Например, к чему приведет попытка чтения несуществующего файла? Или если файл был случайно удален пока программа работала? Такие ситуации обрабатываются при помощи исключений.

Если же Python не может понять, как обойти сложившуюся ситуацию, то ему не остается ничего кроме как поднять руки и сообщить, что обнаружил ошибку. В общем, исключения необходимы, чтобы сообщать программисту об ошибках.

In [4]:
100 / 0

ZeroDivisionError: division by zero

In [ ]:
2 + '3'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
float('abc')

ValueError: could not convert string to float: 'abc'

В этих двух примерах генерируются исключения TypeError и ValueError соответственно. Подсказки дают нам полную информацию о том, где порождено исключение, и с чем оно связано.

Рассмотрим иерархию встроенных в python исключений, хотя иногда вам могут встретиться и другие, так как программисты могут создавать собственные исключения.+ BaseException - базовое исключение, от которого берут начало все остальные.+ SystemExit - исключение, порождаемое функцией sys.exit при выходе из программы.+ KeyboardInterrupt - порождается при прерывании программы пользователем (обычно сочетанием клавиш Ctrl+C).+ GeneratorExit - порождается при вызове метода close объекта generator.+ Exception - а вот тут уже заканчиваются полностью системные исключения (которые лучше не трогать) и начинаются обыкновенные, с которыми можно работать.

+ StopIteration - порождается встроенной функцией next, если в итераторе больше нет элементов.
+ ArithmeticError - арифметическая ошибка.
    + FloatingPointError - порождается при неудачном выполнении операции с плавающей запятой. На практике встречается нечасто.
    + OverflowError - возникает, когда результат арифметической операции слишком велик для представления. Не появляется при обычной работе с целыми числами (так как pyth+ поддерживает длинные числа), но может возникать в некоторых других случаях.
    + ZeroDivisionError - деление на ноль.
+ AssertionError - выражение в функции assert ложно.
+ AttributeError - объект не имеет данного атрибута (значения или метода).
+ BufferError - операция, связанная с буфером, не может быть выполнена.
+ EOFError - функция наткнулась на конец файла и не смогла прочитать то, что хотела.
+ ImportError - не удалось импортирование модуля или его атрибута.
+ LookupError - некорректный индекс или ключ.
    + IndexError - индекс не входит в диапазон элементов.
    + KeyError - несуществующий ключ (в словаре, множестве или другом объекте). 
+ MemoryError - недостаточно памяти.
+ NameError - не найдено переменной с таким именем.
    + UnboundLocalError - сделана ссылка на локальную переменную в функции, но переменная не определена ранее. 
+ OSError - ошибка, связанная с системой.
    + BlockingIOError
    + ChildProcessError - неудача при операции с дочерним процессом.
    + ConnectionError - базовый класс для исключений, связанных с подключениями.
        + BrokenPipeError
        + ConnectionAbortedError
        + ConnectionRefusedError
        + ConnectionResetError
    + FileExistsError - попытка создания файла или директории, которая уже существует.
    + FileNotFoundError - файл или директория не существует.
    + InterruptedError - системный вызов прерван входящим сигналом.
    + IsADirectoryError - ожидался файл, но это директория.
    + NotADirectoryError - ожидалась директория, но это файл.
    + PermissionError - не хватает прав доступа.
    + ProcessLookupError - указанного процесса не существует.
    + TimeoutError - закончилось время ожидания.
+ ReferenceError - попытка доступа к атрибуту со слабой ссылкой.
+ RuntimeError - возникает, когда исключение не попадает ни под одну из других категорий.
+ NotImplementedError - возникает, когда абстрактные методы класса требуют переопределения в дочерних классах.
+ SyntaxError - синтаксическая ошибка.
    + IndentationError - неправильные отступы.
        + TabError - смешивание в отступах табуляции и пробелов.
+ SystemError - внутренняя ошибка.
+ TypeError - операция применена к объекту несоответствующего типа.
+ ValueError - функция получает аргумент правильного типа, но некорректного значения.
+ UnicodeError - ошибка, связанная с кодированием / раскодированием unicode в строках.
    + UnicodeEncodeError - исключение, связанное с кодированием unicode.
    + UnicodeDecodeError - исключение, связанное с декодированием unicode.
    + UnicodeTranslateError - исключение, связанное с переводом unicode.
+ Warning - предупреждение.

Для обработки исключений используется конструкция try - except

In [5]:
try:
    k = 1 / 2
except ZeroDivisionError:
    k = 0

print(k)

0.5


В блоке try мы выполняем инструкцию, которая может породить исключение, а в блоке except мы перехватываем их. При этом перехватываются как само исключение, так и его потомки. Например, перехватывая ArithmeticError, мы также перехватываем FloatingPointError, OverflowError и ZeroDivisionError.

In [7]:
try:
    k = 1 / 0
except ArithmeticError:
    k = 0

print(k)

0


Также возможна инструкция except без аргументов, которая перехватывает вообще всё (и прерывание с клавиатуры, и системный выход и т. д.). Поэтому в такой форме инструкция except практически не используется, а используется except Exception. Однако чаще всего перехватывают исключения по одному, для упрощения отладки (вдруг вы ещё другую ошибку сделаете, а except её перехватит).

Ещё две инструкции, относящиеся к нашей проблеме, это finally и else. Finally выполняет блок инструкций в любом случае, было ли исключение, или нет (применима, когда нужно непременно что-то сделать, к примеру, закрыть файл). Инструкция else выполняется в том случае, если исключения не было.

In [7]:
f = open('test.txt')
ints = []
try:
    for line in f:
        ints.append(int(line))
except ValueError:
    print('Это не число. Выходим.')
except Exception:
    print('Это что ещё такое?')
else:
    print('Всё хорошо.')
finally:
    f.close()
    print('Я закрыл файл.')

Это не число. Выходим.
Я закрыл файл.


In [ ]:
x = 3
y = 0

logging.info(f"The values of x and y are {x} and {y}.")
try:
    x / y
    logging.info(f"x / y successful with result: {x / y}.")
except ZeroDivisionError as err:
    logging.error("ZeroDivisionError")

In [10]:
x_vals = [2, 3, 6, 4, 10]
y_vals = [5, 7, 12, 0, 1]

for x_val, y_val in zip(x_vals, y_vals):
    x, y = x_val, y_val
    logging.info(f"The values of x and y are {x} and {y}.")
    try:
        x / y
        logging.info(f"x / y successful with result: {x / y}.")
    except ZeroDivisionError as err:
        logging.exception("ZeroDivisionError")

Настройка логирования с помощью пользовательских логгеров, обработчиков и форматировщиков

In [11]:
import logging

logger2 = logging.getLogger(__name__)
logger2.setLevel(logging.INFO)

handler2 = logging.FileHandler(f"{__name__}.log", mode='w')
formatter2 = logging.Formatter("%(name)s %(asctime)s %(levelname)s %(message)s")

handler2.setFormatter(formatter2)
logger2.addHandler(handler2)

logger2.info(f"Testing the custom logger for module {__name__}...")

In [ ]:
import logging


def test_division(x,y):
    try:
        x / y
        logger2.info(f"x / y successful with result: {x / y}.")
    except ZeroDivisionError as err:
        logger2.exception("ZeroDivisionError")

py_logger = logging.getLogger(__name__)
py_logger.setLevel(logging.INFO)

py_handler = logging.FileHandler(f"{__name__}.log", mode='w')
py_formatter = logging.Formatter("%(name)s %(asctime)s %(levelname)s %(message)s")

py_handler.setFormatter(py_formatter)
py_logger.addHandler(py_handler)

py_logger.info(f"Testing the custom logger for module {__name__}...")

x_vals = [2, 3, 6, 4, 10]
y_vals = [5, 7, 12, 0, 1]

for x_val, y_val in zip(x_vals, y_vals):
    x, y = x_val, y_val
    test_division(x, y)
    py_logger.info(f"Call test_division with args {x} and {y}")

Сначала мы получаем логгер и задаём уровень логирования. Команда logging.getLogger(name) возвращает логгер с заданным именем в том случае, если он существует. В противном случае она создаёт логгер с заданным именем. На практике имя логгера устанавливают с использованием специальной переменной name, которая соответствует имени модуля. Мы назначаем объект логгера переменной. Затем мы, используя команду logging.setLevel(level), устанавливаем нужный нам уровень логирования.

Далее мы настраиваем обработчик. Так как мы хотим записывать сведения о событиях в файл, мы пользуемся FileHandler. Конструкция logging.FileHandler(filename) возвращает объект обработчика файла. Помимо имени лог-файла, можно, что необязательно, задать режим работы с этим файлом. В данном примере режим (mode) установлен в значение write. Есть и другие обработчики, например — StreamHandler, HTTPHandler, SMTPHandler.

Затем мы создаём объект форматировщика, используя конструкцию logging.Formatter(format). В этом примере мы помещаем имя логгера (строку) в начале форматной строки, а потом идёт то, чем мы уже пользовались ранее при оформлении сообщений.

Потом мы добавляем форматировщик к обработчику, пользуясь конструкцией вида <handler>.setFormatter(<formatter>). А в итоге добавляем обработчик к объекту логгера, пользуясь конструкцией <logger>.addHandler(<handler>).

#### Рекомендации по организации логирования в Python

+ Устанавливайте оптимальный уровень логирования. Логи полезны лишь тогда, когда их можно использовать для отслеживания важных ошибок, которые нужно исправлять. Подберите такой уровень логирования, который соответствует специфике конкретного приложения. Вывод в лог сообщений о слишком большом количестве событий может быть, с точки зрения отладки, не самой удачной стратегией. Дело в том, что при таком подходе возникнут сложности с фильтрацией логов при поиске ошибок, которым нужно немедленно уделить внимание.

+ Конфигурируйте логгеры на уровне модуля. Когда вы работаете над приложением, состоящим из множества модулей — вам стоит задуматься о том, чтобы настроить свой логгер для каждого модуля. Установка имени логгера в name помогает идентифицировать модуль приложения, в котором имеются проблемы, нуждающиеся в решении.

+ Включайте в состав сообщений логов отметку времени и обеспечьте единообразное форматирование сообщений. Всегда включайте в сообщения логов отметки времени, так как они полезны в деле поиска того момента, когда произошла ошибка. Единообразно форматируйте сообщения логов, придерживаясь одного и того же подхода в разных модулях.

+ Применяйте ротацию лог-файлов ради упрощения отладки. При работе над большим приложением, в состав которого входит несколько модулей, вы, вполне вероятно, столкнётесь с тем, что размер ваших лог-файлов окажется очень большим. Очень длинные логи сложно просматривать в поисках ошибок. Поэтому стоит подумать о ротации лог-файлов. Сделать это можно, воспользовавшись обработчиком RotatingFileHandler, применив конструкцию, которая строится по следующей схеме: ```logging.handlers.RotatingFileHandler(filename, maxBytes, backupCount)```. Когда размер текущего лог-файла достигнет размера maxBytes, следующие записи будут попадать в другие файлы, количество которых зависит от значения параметра backupCount. Если установить этот параметр в значение K — у вас будет K файлов журнала.